In [1]:
import model_backend as mdc
from pathlib import Path
import xarray as xr
import rioxarray as rioxr 
import numpy as np 

In [4]:
def make_test_data(tmp_path) -> xr.Dataset:
    # Create proper geographical coordinates
    lon = np.linspace(-180, 180, 10)
    lat = np.linspace(-90, 90, 10)
    data = np.random.rand(10, 10)
    
    # Create dataset with proper dimension order for geospatial data
    ds = xr.Dataset(
        data_vars={
            "temp": (("y", "x"), data),
        },
        coords={
            "y": lat,
            "x": lon,
        },
    )
    
    # Save to netCDF
    ds.to_netcdf(tmp_path / "test_data.nc", mode="w", format="NETCDF4")
    
    
    # First load with netCDF engine
    with xr.open_dataset(tmp_path / "test_data.nc") as data:
    
        # Properly set spatial attributes needed by rioxarray - this is what makes .rio work
        data.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=True)
        data.rio.write_crs("EPSG:4326", inplace=True)
        ds = data.copy()
        
    return ds

In [5]:
ds = make_test_data(Path("./"))
ds

<xarray.Dataset> Size: 968B
Dimensions:      (y: 10, x: 10)
Coordinates:
  * y            (y) float64 80B -90.0 -70.0 -50.0 -30.0 ... 30.0 50.0 70.0 90.0
  * x            (x) float64 80B -180.0 -140.0 -100.0 ... 100.0 140.0 180.0
    spatial_ref  int64 8B 0
Data variables:
    temp         (y, x) float64 800B ...

In [6]:

# Save to netCDF

# First load with netCDF engine
with xr.open_dataset("test_data.nc") as ds:

    print(ds.rio.crs)  # Check if CRS is set
    print(ds)
    # Properly set spatial attributes needed by rioxarray - this is what makes .rio work
    # ds.rio.set_spatial_dims(x_dim="x", y_dim="y", inplace=True)
    ds.rio.write_crs("EPSG:4326", inplace=True)
    ds

None
<xarray.Dataset> Size: 960B
Dimensions:  (y: 10, x: 10)
Coordinates:
  * y        (y) float64 80B -90.0 -70.0 -50.0 -30.0 ... 30.0 50.0 70.0 90.0
  * x        (x) float64 80B -180.0 -140.0 -100.0 -60.0 ... 100.0 140.0 180.0
Data variables:
    temp     (y, x) float64 800B ...


In [7]:
ds

<xarray.Dataset> Size: 968B
Dimensions:      (y: 10, x: 10)
Coordinates:
  * y            (y) float64 80B -90.0 -70.0 -50.0 -30.0 ... 30.0 50.0 70.0 90.0
  * x            (x) float64 80B -180.0 -140.0 -100.0 ... 100.0 140.0 180.0
    spatial_ref  int64 8B 0
Data variables:
    temp         (y, x) float64 800B ...

In [8]:
ds.rio.crs

CRS.from_wkt('GEOGCS["WGS 84",DATUM["WGS_1984",SPHEROID["WGS 84",6378137,298.257223563,AUTHORITY["EPSG","7030"]],AUTHORITY["EPSG","6326"]],PRIMEM["Greenwich",0,AUTHORITY["EPSG","8901"]],UNIT["degree",0.0174532925199433,AUTHORITY["EPSG","9122"]],AXIS["Latitude",NORTH],AXIS["Longitude",EAST],AUTHORITY["EPSG","4326"]]')

In [9]:
ds.rio.write_crs(None, inplace=True)

<xarray.Dataset> Size: 968B
Dimensions:      (y: 10, x: 10)
Coordinates:
  * y            (y) float64 80B -90.0 -70.0 -50.0 -30.0 ... 30.0 50.0 70.0 90.0
  * x            (x) float64 80B -180.0 -140.0 -100.0 ... 100.0 140.0 180.0
    spatial_ref  int64 8B 0
Data variables:
    temp         (y, x) float64 800B ...

# Build model instance

data is assumed to live in a directory: 

data/in

relative to the base directory of the package

In [ ]:
path = Path.cwd().parent / "data" / "in"
path

In [ ]:
model = mdc.JModel(
    input = path / "ERA5land_global_t2m_dailyStats_mean_01Deg_2024_08_data.nc", 
    output = Path.cwd().parent / 'data' / "out" / "output_map.nc", 
    r0_path= path / "R0_pip_stats.csv",
    run_mode="forbidden", 
    grid_data_baseurl="https://gisco-services.ec.europa.eu/distribution/v2/nuts",
    nuts_level =3, 
    resolution = "10M", 
    year=2024, 
)

# Run model -> load data -> apply model transformations -> save back to netcdf

In [ ]:
model.run()

# Have a look at produced data

In [ ]:
data = xr.open_dataset(
            Path.cwd().parent / 'data' / "out" / "output_map.nc", engine="rasterio"
        )

In [ ]:
data

In [ ]:
data["r0"].mean(dim="valid_time").plot(cmap="viridis")